In [ ]:
import variational_bayes as vb
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Define parameters
np.random.seed(1)
n, p = 100, 3
regression_precision = 5
measurement_precision = 10


# Generate data
features = np.random.normal(0, 1, (n, p))
measurements = features + np.random.normal(0, 1, (n, p)) / np.sqrt(measurement_precision)
coefficients = np.random.normal(0, 1, p)
predictor = np.dot(features, coefficients)
y = predictor + np.random.normal(0, 1, n) / np.sqrt(regression_precision)

# Regression with exactly measured covariates

In [ ]:
factors = {
    'coefficients': vb.MultiNormalDistribution(np.zeros(p), np.eye(p)),
    'precision': vb.GammaDistribution(1, 1)
}
likelihoods = [
    vb.LinearRegressionDistribution(features, factors['coefficients'], factors['precision']).likelihood(y)
]
model = vb.Model(factors, likelihoods)
model.update(10)

In [ ]:
vb.plot_proba(factors['precision'], reference=regression_precision)

In [ ]:
vb.plot_comparison(factors['coefficients'], coefficients)

# Regression with noisy covariates

In [ ]:
factors = {
    'coefficients': vb.MultiNormalDistribution(np.zeros(p), np.eye(p)),
    'regression_precision': vb.GammaDistribution(1, 1),
    'features': vb.MultiNormalDistribution(np.zeros((n, p)), np.eye(p) * np.ones((n, 1, 1))),
    # 'measurement_precision': vb.GammaDistribution(1, 1),
}
likelihoods = [
    vb.LinearRegressionDistribution(
        factors['features'], factors['coefficients'], factors['regression_precision']
    ).likelihood(y),
    vb.NormalDistribution(factors['features'], measurement_precision).likelihood(measurements)
]
model = vb.Model(factors, likelihoods)
model.update(200)
pass

In [ ]:
vb.plot_proba(factors['regression_precision'], reference=regression_precision)

In [ ]:
vb.plot_comparison(factors['features'], features, marker='.')

In [ ]:
vb.plot_comparison(factors['coefficients'], coefficients, marker='.')

In [ ]:
for i in range(p):
    print(i)
    print("Raw measurement correlation", np.corrcoef(measurements[:, i], features[:, i], rowvar=False)[0, 1])
    print("Inferred features correlation", np.corrcoef(factors['features'].mean[:, i], features[:, i], rowvar=False)[0, 1])
    print()

# Regression with noisy covariates incorrectly assumed to be exact

In [ ]:
factors = {
    'coefficients': vb.MultiNormalDistribution(np.zeros(p), np.eye(p)),
    'precision': vb.GammaDistribution(1, 1)
}
likelihoods = [
    vb.LinearRegressionDistribution(measurements, factors['coefficients'], factors['precision']).likelihood(y)
]
model = vb.Model(factors, likelihoods)
model.update(10)

In [ ]:
vb.plot_proba(factors['precision'], reference=regression_precision)

In [ ]:
vb.plot_comparison(factors['coefficients'], coefficients, marker='.')